In [3]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [1]:
def create_filenames(letter, number):
    
    c = chr(ord('0') + number + 1)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + number + '-sell.csv'
    str_spl = 'MS-' + letter + number + '-supply.csv'
    return str_i, str_sl, str_spl

Количество магазинов в штате

In [2]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

Для магазина b1

In [21]:
stor_daily_b1 = pd.DataFrame(columns=['date', 'apple', 'pen'])

length_sell = len(sell_upg_b1) - 1
length_spl = len(spl_b1) - 1

curr_apple = 0
curr_pen = 0

for i in range(length_sell):
    
    curr_date = sell_upg_b1['date'][i]
    for j in range(length_spl):
        if spl_b1['date'][j] == curr_date:
            curr_apple = curr_apple + spl_b1['apple'][j]
            curr_pen = curr_pen + spl_b1['pen'][j]
            
    curr_apple = curr_apple - sell_upg_b1['apple'][i]
    curr_pen  = curr_pen - sell_upg_b1['pen'][i]
   
    stor_daily_b1.at[i, 'apple'] = curr_apple
    stor_daily_b1.at[i, 'pen'] = curr_pen
    stor_daily_b1.at[i, 'date'] = curr_date
    
#stor_daily_b1.head(n=15)

,date,apple,pen
0,2006-01-01,33271,2574
1,2006-01-02,31409,2431
2,2006-01-03,29529,2260
3,2006-01-04,27732,2107
4,2006-01-05,25790,1974
5,2006-01-06,23892,1825
6,2006-01-07,21997,1687
7,2006-01-08,20161,1567
8,2006-01-09,18293,1442
9,2006-01-10,16414,1286


In [ ]:
def daily_stor(state):
    
    max = num_shop(state)
    files = {}
    path = {}
    answer = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob(os.getcwd() + '/input/*'+ state + c + '*.csv')
        for j in range(0, len(path[shop])):
            files[shop][j]= pd.read_csv(path[shop][j])
        i = i + 1
    INV = 0
    SELL = 1
    SPL = 2
    
    shop = 0
    while shop < max:
        files[shop][SELL]['sku_num'] = files[shop][SELL]['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
        daily = pd.crosstab(files[shop][SELL]['date'], files[shop][SELL]['sku_num'], margins = True)
        daily_new = pd.DataFrame(columns=['date', 'apple_sell', 'pen_sell'])
        daily_new['date'] = daily.index
        daily_new['apple_sell'] = daily[0].values
        daily_new['pen_sell'] = daily[1].values
        more_inf = pd.merge(daily_new, files[shop][SPL], how='outer', on='date')
        more_inf['apple'].fillna(method='ffill')
        more_inf['pen'].fillna(method='ffill')
        more_inf['apple'] = more_inf['apple'] - more_inf['apple_sell']
        more_inf['pen'] = more_inf['pen'] - more_inf['pen_sell']
        more_inf.drop(columns=['pen_sell', 'apple_sell'])
        name = create_name(state, shop + 1)
        more_inf.to_csv('answer/' + name, index=False)
        answer[shop] = more_inf
        shop = shop + 1
        
    return answer
    

In [5]:
def main():
    
    daily_stor('b')
    daily_stor('m')
    daily_stor('s')
    
main() 
    

['/home/varvara/6_sem_prac/task_3/input/MS-s5-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-m2-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-b2-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s4-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s2-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s1-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s3-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-b1-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-m1-sell.csv']
